In [1]:
%load_ext cython
import matplotlib.pyplot as plt
import meshpy.tet
import numpy as np
#%load_ext snakeviz

In [45]:
11 // 10

1

In [136]:
%%cython -a
import numpy as np
def build_aabb(int nx, int ny, int nz, long [:,:] tetra, double [:,:] nodes, long [:,:] aabb, double [:] step):
    cdef int i, ntetra, j, ix, iy, iz
    cdef double [:,:] bb = np.zeros((2,3))
#     cdef double [:] maxs = np.zeros(3)
    cdef double [:,:] points = np.zeros((4,3))
    ntetra = tetra.shape[0]
    # create a mask for extracting corners from 
    cdef long [:,:] corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
                                        [0, 0, 1, 0, 0, 1, 1, 1],
                                        [0, 0, 0, 1, 1, 1, 0, 1]]).astype(int)
    
    cdef long [:] tetra_count= np.zeros(nx*ny*nz).astype(int)
    for i in range(ntetra):
        # copy points into tmp array
        for k in range(4):
            for l in range(3):
                points[k,l] = nodes[tetra[i,k],l]
        for l in range(3):
            bb[0,l] = np.min(points[:,l])#,axis=0)
            bb[1,l] = np.max(points[:,l])#,axis=0)
#             print(bb[0,l],bb[1,l])
        # check which corners of bounding box are inside a cell 
        for j in range(8):
            ix = int(bb[corners[0,j],0] // step[0]) 
            iy = int(bb[corners[1,j],1] // step[1])
            iz = int(bb[corners[2,j],2] // step[2]) 
            aabb[ix+nx*iy+nx*ny*iz,tetra_count[ix+nx*iy+nx*ny*iz]] = i
            tetra_count[ix+nx*iy+nx*ny*iz]+=1


    return

In [137]:
%%cython -a
cdef stp(double [:] v1, double [:] v2, double [:] v3):
    cdef double [3] c

    s = 0.
    c[0] = v1[1]*v2[2]-v1[2]*v2[1]
    c[1] = v1[2]*v2[0]-v1[0]*v2[2]
    c[2] = v1[0]*v2[1]-v1[1]*v2[0]
    s = c[0]*v3[0] + c[1]*v3[1] + c[2]*v3[2]
    return s
import numpy as np
def find_tetras(double [:,:] nodes, long [:,:] tetras, long [:,:] aabbindx, double [:,:] pos, 
               double [:,:] bcs, long [:] tetra_index):
    cdef int npts = pos.shape[0]
    cdef int ntetra 
    cdef int i,j,k, l
    cdef double [3] vap, vbp, vab, vac, vad, vbc, vbd
    cdef double va, vb, vc, vd, v
    cdef double [4] c
    for i in range(npts):
        ntetra = len(aabbindx[i,:])
        for j in range(ntetra):
            # aabbindx should be ordered so, if we get to -1 then skip
            print(aabbindx[i,j])
            if aabbindx[i,j] < 0:
                break
            for k in range(3):
                vap[k] = pos[i,k] - nodes[tetras[aabbindx[i,j],0],k]
                vbp[k] = pos[i,k] - nodes[tetras[aabbindx[i,j],1],k]
                
                vab[k] = nodes[tetras[aabbindx[i,j],1],k] - nodes[tetras[aabbindx[i,j],0],k]
                vac[k] = nodes[tetras[aabbindx[i,j],2],k] - nodes[tetras[aabbindx[i,j],0],k]
                vad[k] = nodes[tetras[aabbindx[i,j],3],k] - nodes[tetras[aabbindx[i,j],0],k]
                vbc[k] = nodes[tetras[aabbindx[i,j],2],k] - nodes[tetras[aabbindx[i,j],1],k]
                vbd[k] = nodes[tetras[aabbindx[i,j],3],k] - nodes[tetras[aabbindx[i,j],1],k]
                
            va = 0
            vb = 0
            vc = 0
            vd = 0
            v = 0
            # calculate scalar triple product as each bc coordinate
            # if any coordinte is 0 then this tetra is not the right one so keep looking
            v = stp(vac,vad,vab)
            v /= 6.
            if v <= 0:
                print('v < 0')
                continue
            va = stp(vbd,vbc,vbp)
            va /= 6.
            va /= v
            if va < 0:
                print('va < 0')
                continue
            vb = stp(vac,vad,vap)
            vb /= 6.
            vb /= v
            if vb < 0:
                print('vb <0')
                continue
            vc = stp(vad,vab,vap)
            vc /= 6.
            vc /= v
            if vc < 0:
                print('vc < 0')
                continue
            vd = stp(vab,vac,vap)
            vd /= 6.
            vd /= v
            if vd < 0:
                print('vd < 0')
                continue

            # all three coordinates should be > 0
            tetra_index[i] = aabbindx[i,j]
            bcs[i,0] = va
            bcs[i,1] = vb
            bcs[i,2] = vc
            bcs[i,3] = vd
            

                
#                 vab = points[:, :, 1, :] - points[:, :, 0, :]
#                 vac = points[:, :, 2, :] - points[:, :, 0, :]
#                 vad = points[:, :, 3, :] - points[:, :, 0, :]
#                 vbc = points[:, :, 2, :] - points[:, :, 1, :]
#                 vbd = points[:, :, 3, :] - points[:, :, 1, :]
#             vap = pos[i,:] - 
#         vap = pos[i,:] - nodes[] 
#         vap = pos[:,None,:] - points[:, :, 0, :]
#         vbp = pos[:,None,:] - points[:, :, 1, :]
#         # vcp = p - points[:, 2, :]
#         # vdp = p - points[:, 3, :]
#         vab = points[:, :, 1, :] - points[:, :, 0, :]
#         vac = points[:, :, 2, :] - points[:, :, 0, :]
#         vad = points[:, :, 3, :] - points[:, :, 0, :]
#         vbc = points[:, :, 2, :] - points[:, :, 1, :]
#         vbd = points[:, :, 3, :] - points[:, :, 1, :]
        
#         va = np.einsum('ikj, iklj->ikl', vbp, np.cross(vbd, vbc, axisa=2, axisb=2)) / 6.
#         vb = np.einsum('ikj, iklj->ikl', vap, np.cross(vac, vad, axisa=2, axisb=2)) / 6.
#         vc = np.einsum('ikj, iklj->ikl', vap, np.cross(vad, vab, axisa=2, axisb=2)) / 6.
#         vd = np.einsum('ikj, iklj->ikl', vap, np.cross(vab, vac, axisa=2, axisb=2)) / 6.
#         v = np.einsum('ikj, iklj->ikl', vab, np.cross(vac, vad, axisa=2, axisb=2)) / 6.
#         c = np.zeros((va.shape[0],va.shape[1],4))

In [153]:
class TetMesh2:
    def __init__(self,origin,maximum,nodes,tetra,neighbours,nx=2,ny=2,nz=2):
        self.origin = np.array(origin)
        self.maximum = np.array(maximum)
        self.nodes = nodes
        self.tetra = tetra
        self.ntetra = tetra.shape[0]

        self.neighbours = neighbours
        self.step = (self.maximum-self.origin)/(np.array([nx,ny,nz]))
        print(self.step)
        self.aabb = np.zeros(((nx+1)*(ny+1)*(nz+1),self.ntetra)).astype(int)

        self.nx = nx 
        self.nz = nz 
        self.ny = ny 
        
#         nx -= 1
#         ny -=1 
#         nz-=1
        self.aabb[:] = -1

        self._build_aabb()
        self.corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
                                      [0, 0, 1, 0, 0, 1, 1, 1],
                                      [0, 0, 0, 1, 1, 1, 0, 1]]).astype(int)
    def _build_aabb(self):
        build_aabb(self.nx+1,self.ny+1,self.nz+1,self.tetra,self.nodes-0.5*self.step,self.aabb,self.step)
        bb = np.zeros((3,2))
        idx = np.arange(0,self.aabb.shape[0])
        xi,yi,zi = self.global_aabb_index_to_local(idx[np.all(self.aabb == -1,axis=1)])
        bb = np.zeros((3,2))
        for i in range(xi.shape[0]):
            bb[0,0] = xi[i]*self.step[0] - self.step[0]*0.5
            bb[0,1] = (xi[i]+1)*self.step[0] - self.step[0]*0.5

            bb[1,0] = yi[i]*self.step[1] - self.step[1]*0.5
            bb[1,1] = (yi[i]+1)*self.step[1] - self.step[1]*0.5

            bb[2,0] = zi[i]*self.step[2] - self.step[2]*0.5
            bb[2,1] = (zi[i]+1)*self.step[2] - self.step[2]*0.5

            corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
                                                [0, 0, 1, 0, 0, 1, 1, 1],
                                                [0, 0, 0, 1, 1, 1, 0, 1]]).astype(int)
            corner_loc = np.zeros((8,3))
            corner_loc[:,0] = bb[0,corners[0,:]]#bb[corners]
            corner_loc[:,1] = bb[1,corners[1,:]]
            corner_loc[:,2] = bb[2,corners[2,:]]
            for j, t in enumerate(self.tetra):
                points = self.nodes[t] - self.step*0.5
                vap = corner_loc[:,None,:] - points[None,  0, :]
                vbp = corner_loc[:,None,:] - points[None, 1, :]
        #         # vcp = p - points[:, 2, :]
        #         # vdp = p - points[:, 3, :]
                vab = points[None, 1, :] - points[None, 0, :]
                vac = points[None, 2, :] - points[None, 0, :]
                vad = points[None, 3, :] - points[None, 0, :]
                vbc = points[None, 2, :] - points[None, 1, :]
                vbd = points[None, 3, :] - points[None, 1, :]

                va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1)) / 6.
                vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1)) / 6.
                vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1)) / 6.
                vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1)) / 6.
                v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1)) / 6.

                c = np.zeros((4,8))
                c[0,:] = va / v
                c[1,:] = vb / v
                c[2,:] = vc / v
                c[3,:] = vd / v
#                 print(np.all(c>0,axis=0))
                if np.any(np.all(c>0,axis=0)):
#                     print('All points inside a tetra')
#                     print(self.aabb[xi[i]+yi[i]*self.nx+zi[i]*self.nx*self.ny,0])
                    self.aabb[xi[i]+yi[i]*self.nx+zi[i]*self.nx*self.ny,0] = j
#                     print(self.aabb[xi[i]+yi[i]*self.nx+zi[i]*self.nx*self.ny,0])
                    break
#                 if j == self.
        #         print(mesh2.calc_bary(corner_loc,t))

        #         points = mesh.nodes[t]
        #         print(points)
                if j == len(self.tetra)-1:
                    print(corner_loc)
                    print('Couldnt find any tetra')
    
    
        # now sort
    def get_tetra2(self,pos):
        pos = np.array(pos) - self.step*.5
        tetras = np.zeros(pos.shape[0]).astype(int)
        bcs = np.zeros((pos.shape[0],4))
        for i in range(pos.shape[0]):
            aabbidx = self.pos_to_aabb_index(pos)
            cells = self.aabb[aabbidx[i],:]
            cells=cells[cells>=0]
            npts = len(cells)
            points = self.nodes[self.tetra[cells]]
            vap = pos[i,:] - points[:, 0, :]
            vbp = pos[i,:] - points[:, 1, :]
            # vcp = p - points[:, 2, :]
            # vdp = p - points[:, 3, :]
            vab = points[:, 1, :] - points[:, 0, :]
            vac = points[:, 2, :] - points[:, 0, :]
            vad = points[:, 3, :] - points[:, 0, :]
            vbc = points[:, 2, :] - points[:, 1, :]
            vbd = points[:, 3, :] - points[:, 1, :]

            va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1), axis=1) / 6.
            vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
            vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1), axis=1) / 6.
            vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1), axis=1) / 6.
            v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
            
            c = np.zeros((4, npts))
            c[0, :] = va / v
            c[1, :] = vb / v
            c[2, :] = vc / v
            c[3, :] = vd / v
#             print(c)
            tetras[i] = cells[np.all(c>0,axis=0)]
            #             print(c)
            bcs[i,:] = c[:,np.all(c>0,axis=0)].T
            
        return tetras,bcs
    
    def calc_bary(self,pos,tetra):
            points = self.nodes[self.tetra[tetra]]
            vap = pos - points[:, 0, :]
            vbp = pos - points[:, 1, :]
            # vcp = p - points[:, 2, :]
            # vdp = p - points[:, 3, :]
            vab = points[:, 1, :] - points[:, 0, :]
            vac = points[:, 2, :] - points[:, 0, :]
            vad = points[:, 3, :] - points[:, 0, :]
            vbc = points[:, 2, :] - points[:, 1, :]
            vbd = points[:, 3, :] - points[:, 1, :]

            va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1), axis=1) / 6.
            vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
            vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1), axis=1) / 6.
            vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1), axis=1) / 6.
            v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
            
            c = np.zeros((4))
            c[0] = va / v
            c[1] = vb / v
            c[2] = vc / v
            c[3] = vd / v
            
            return c
    def get_tetrac(self,pos):
        aabbidx = self.pos_to_aabb_index(pos)
        # get the tetras for each aabb cell where our points are -1 are tetras which 
        # aren't seen in the cell'
        tetra_index = np.zeros(pos.shape[0]).astype(int)
        tetra_index[:] = -1
        bcs = np.zeros((pos.shape[0],4))
        cells = self.aabb[aabbidx,:]
        find_tetras(self.nodes, self.tetra, cells, pos,bcs,tetra_index)
        return tetra_index, bcs
    def get_tetra(self,pos):
        pos = np.array(pos)
        
        # create containers to store the results (tetra and the bc for each point)
        tetras = np.zeros(pos.shape[0]).astype(int)
        tetras[:] = -1
        bcs = np.zeros((pos.shape[0],4))
        
        # find the aabb cells for each point
        aabbidx = self.pos_to_aabb_index(pos)
        # get the tetras for each aabb cell where our points are -1 are tetras which 
        # aren't seen in the cell
        cells = self.aabb[aabbidx,:]
        
        # create a in array the same shape as cells array
        pos_idx = np.tile(np.arange(cells.shape[0]).T,(cells.shape[1],1)).astype(int).T
    
        # slice the pos and cells arrays for only the tetras that are inside out aabb cells
        # this will create a flat array of points and tetras
        pos2 = pos[pos_idx[cells>0]]
        cells2 = cells[cells>0]
        npts = pos2.shape[0]
        # now calculate the barycentric coordinates for all of these points
        points = self.nodes[self.tetra[cells2]]
        vap = pos2 - points[:, 0, :]
        vbp = pos2 - points[:, 1, :]
        # vcp = p - points[:, 2, :]
        # vdp = p - points[:, 3, :]
        vab = points[:, 1, :] - points[:, 0, :]
        vac = points[:, 2, :] - points[:, 0, :]
        vad = points[:, 3, :] - points[:, 0, :]
        vbc = points[:, 2, :] - points[:, 1, :]
        vbd = points[:, 3, :] - points[:, 1, :]

        va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1), axis=1) / 6.
        vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
        vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1), axis=1) / 6.
        vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1), axis=1) / 6.
        v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
        
        c = np.zeros((4, npts))
        c[0, :] = va / v
        c[1, :] = vb / v
        c[2, :] = vc / v
        c[3, :] = vd / v
        mask = np.zeros(cells.shape).astype(bool)
        mask[cells>0] = np.all(c>0,axis=0)
#         mask= np.all(c>0,axis=0)
#         print(mask[0])

#         print('pi',pos_idx[cells>0][np.all(c>0,axis=0)])
        tetras[np.sum(mask,axis=1).astype(bool)] = cells[mask]
#         bc[np.sum(mask,axis=1).astype(bool),:] = c[:,mask].T
        return tetras#,bc

#         return self.aabb[aabbidx,:]
    def pos_to_aabb_index(self,pos):
        ix = pos[:, 0] - self.origin[None, 0]
        iy = pos[:, 1] - self.origin[None, 1]
        iz = pos[:, 2] - self.origin[None, 2]
        ix = ix // self.step[None, 0]
        iy = iy // self.step[None, 1]
        iz = iz // self.step[None, 2]
        return ix.astype(int)+iy.astype(int)*self.nx+iz.astype(int)*self.nx*self.ny
    
    def global_aabb_index_to_local(self,global_index):
        global_index = np.array(global_index)
        x_index = global_index % self.nx
        y_index = global_index // self.nx % \
                  self.ny
        z_index = global_index // self.nx // \
                  self.ny
        
        return x_index, y_index, z_index
#         corner_pos = self.origin[:,None]+self.step[:,None]*np.vstack([xcorners,ycorners,zcorners])
#         print(corner_pos)

In [154]:
def build(origin,maximum,n_tetra,nx,ny,nz):
    minx = origin[0]
    miny = origin[1]
    minz = origin[2]
    maxx = maximum[0]
    maxy =  maximum[1]
    maxz =  maximum[2]
    points = np.array([
        (minx, miny, minz),
        (maxx, miny, minz),
        (maxx, maxy, minz),
        (minx, maxy, minz),
        (minx, miny, maxz),
        (maxx, miny, maxz),
        (maxx, maxy, maxz),
        (minx, maxy, maxz)
    ])
#         self.surfaces['top'] = (
#             (minx, miny, maxz), (maxx, miny, maxz), (maxx, maxy, maxz),
#             (minx, maxy, maxz))
#         self.surfaces['bottom'] = (
#             (minx, miny, minz), (maxx, miny, minz), (maxx, maxy, minz),
#             (minx, maxy, minz))

#         self.points = points
#         # calculate the 3 principal components to find the local coordinate
#         # system
#         self.pca.fit(points)
#         # project the points into this coordinate system
#         newp = self.pca.transform(points)

    lengthU = maxx - minx
    lengthV = maxy - miny
    lengthW = maxz - minz
    boxVol = lengthU * lengthW * lengthV
    correction_factor = 1.91
    maxvol = correction_factor * boxVol / n_tetra
    facets = [
        [0, 1, 2, 3],
        [4, 5, 6, 7],
        [0, 4, 5, 1],
        [1, 5, 6, 2],
        [2, 6, 7, 3],
        [3, 7, 4, 0]
    ]

    # create the mesh
    info = meshpy.tet.MeshInfo()
    # use the projected points to build the mesh
    info.set_points(points)
    info.set_facets(facets)
    meshpy_mesh = meshpy.tet.build(info, max_volume=maxvol,
                                   options=meshpy.tet.Options('pqn'))
    nodes = np.array(meshpy_mesh.points)
    elements=  np.array(meshpy_mesh.elements, dtype=np.int64)
    neighbours = np.array(meshpy_mesh.neighbors, dtype=np.int64)
    return TetMesh2(origin,maximum,nodes,elements,neighbours,nx,ny,nz)#    def __init__(self,origin,maximum,nodes,tetra,neighbours,nx=10,ny=10,nz=10):


In [155]:
# for i,t in enumerate([100,1000,5000,7000,10000,15000,20000]):#range(1,10):
#     i=i+2
#     print(i*i*i)
x = np.linspace(0.01,.99,20)
y = np.linspace(0.01,.99,20)
z = np.linspace(0.01,.99,20)
xx,yy,zz = np.meshgrid(x,y,z)
points = np.vstack([xx.flatten(),yy.flatten(),zz.flatten()]).T
mesh2 = build([0,0,0],[1,1,1],1000,10,10,10)
#points = np.mean(mesh.nodes[mesh.tetra],axis=1)
#mask = mesh.get_tetra2(points)#np.random.random((100,3)))
# for t in mesh.tetra:
#     points = mesh.nodes[t]
#     try:
#         mesh.get_tetra([np.mean(points,axis=0)])
#     except:
#         print('exception ',i,np.mean(points,axis=0))
#mesh.get_tetra(np.array([[0.69,0.5,.99]]))#np.random.random((100,3)))
# for i in range(100):
#     print(A[i,A[i,:]>0])

[0.1 0.1 0.1]
[[ 0.25 -0.05  0.95]
 [ 0.35 -0.05  0.95]
 [ 0.25  0.05  0.95]
 [ 0.25 -0.05  1.05]
 [ 0.35 -0.05  1.05]
 [ 0.25  0.05  1.05]
 [ 0.35  0.05  0.95]
 [ 0.35  0.05  1.05]]
Couldnt find any tetra
[[ 0.35 -0.05  0.95]
 [ 0.45 -0.05  0.95]
 [ 0.35  0.05  0.95]
 [ 0.35 -0.05  1.05]
 [ 0.45 -0.05  1.05]
 [ 0.35  0.05  1.05]
 [ 0.45  0.05  0.95]
 [ 0.45  0.05  1.05]]
Couldnt find any tetra
[[ 0.45 -0.05  0.95]
 [ 0.55 -0.05  0.95]
 [ 0.45  0.05  0.95]
 [ 0.45 -0.05  1.05]
 [ 0.55 -0.05  1.05]
 [ 0.45  0.05  1.05]
 [ 0.55  0.05  0.95]
 [ 0.55  0.05  1.05]]
Couldnt find any tetra
[[ 0.55 -0.05  0.95]
 [ 0.65 -0.05  0.95]
 [ 0.55  0.05  0.95]
 [ 0.55 -0.05  1.05]
 [ 0.65 -0.05  1.05]
 [ 0.55  0.05  1.05]
 [ 0.65  0.05  0.95]
 [ 0.65  0.05  1.05]]
Couldnt find any tetra
[[ 0.65 -0.05  0.95]
 [ 0.75 -0.05  0.95]
 [ 0.65  0.05  0.95]
 [ 0.65 -0.05  1.05]
 [ 0.75 -0.05  1.05]
 [ 0.65  0.05  1.05]
 [ 0.75  0.05  0.95]
 [ 0.75  0.05  1.05]]
Couldnt find any tetra
[[0.35 0.05 0.95]
 [0.45 0

In [156]:
print(mesh2.aabb[:,0])

[ 17 302 302 ...  79  79  17]


In [157]:
#%%snakeviz
points = np.random.random((10000,3))

In [158]:
aabbidx = mesh2.pos_to_aabb_index(points)
idx = np.arange(0,mesh2.aabb.shape[0])
xi,yi,zi = mesh2.global_aabb_index_to_local(idx[np.all(mesh2.aabb == -1,axis=1)])
# aabbidx[print(aabbidx.shape)]

In [159]:
xi

array([3, 4, 5, 6, 7, 4, 5, 6, 7, 8, 5, 6, 7, 8, 9, 6, 7, 8, 9, 0, 7, 1])

In [160]:
mask2 = mesh2.get_tetrac(points)#np.random.random((100,3)))

101
va < 0
363
vb <0
561
va < 0
589
va < 0
614
va < 0
806
vb <0
-1
12
va < 0
149
vb <0
317
va < 0
351
vb <0
414
va < 0
651
va < 0
800
vb <0
804
vb <0
805
vb <0
851
va < 0
-1
1291
va < 0
-1
18
va < 0
207
vb <0
329
vb <0
726
va < 0
766
va < 0
774
vb <0
-1
40
vc < 0
746
va < 0
854
va < 0
1345
vb <0
-1
126
va < 0
197
vb <0
402
va < 0
405
vc < 0
428
vb <0
440
vb <0
446
va < 0
496
vb <0
507
va < 0
879
vb <0
938
vb <0
978
va < 0
1115
va < 0
1162
va < 0
1238
va < 0
1240
va < 0
1340
va < 0
1349
vb <0
1427
vc < 0
1431
vb <0
1432
vb <0
-1
1285
vb <0
-1
205
vb <0
224
vc < 0
455
va < 0
490
vd < 0
639
vb <0
683
va < 0
934
va < 0
1241
vc < 0
1348
va < 0
-1
1437
-1
182
va < 0
-1
109
va < 0
257
vd < 0
273
va < 0
339
va < 0
504
vb <0
790
vb <0
832
va < 0
835
vb <0
943
va < 0
962
va < 0
966
va < 0
1060
va < 0
1268
va < 0
1275
va < 0
-1
175
vd < 0
984
va < 0
1038
va < 0
1085
va < 0
1186
va < 0
-1
142
va < 0
294
vb <0
1082
vb <0
-1
5
va < 0
59
va < 0
93
va < 0
116
vb <0
785
vb <0
1438
va < 0
1440
va < 0
14

vc < 0
1092
va < 0
1097
va < 0
1099
va < 0
1112
va < 0
1157
vb <0
1220
va < 0
1224
va < 0
1225
vb <0
1227
va < 0
1228
va < 0
-1
15
va < 0
72
vb <0
204
va < 0
245
vb <0
401
va < 0
445
va < 0
714
va < 0
778
va < 0
779
vb <0
839
vb <0
-1
450
va < 0
695
va < 0
1280
vb <0
-1
9
vb <0
188
vb <0
258
vb <0
324
va < 0
344
va < 0
418
vb <0
956
vb <0
1136
vb <0
1139
va < 0
1168
va < 0
1425
vb <0
-1
104
-1
37
vb <0
145
va < 0
286
vb <0
303
va < 0
308
vb <0
353
va < 0
459
va < 0
539
va < 0
743
va < 0
767
va < 0
852
vb <0
1182
va < 0
1184
vb <0
1239
vc < 0
-1
1439
vb <0
-1
1038
va < 0
1119
va < 0
1246
va < 0
-1
1407
vc < 0
-1
648
va < 0
727
va < 0
1036
vc < 0
-1
310
-1
104
vc < 0
-1
111
vb <0
374
va < 0
379
vb <0
686
vb <0
-1
98
va < 0
148
va < 0
172
va < 0
261
va < 0
434
va < 0
479
vb <0
491
vd < 0
549
vb <0
720
va < 0
772
vb <0
814
vc < 0
899
vc < 0
950
va < 0
955
vc < 0
994
vc < 0
1199
vb <0
1261
va < 0
-1
12
va < 0
76
va < 0
207
vb <0
311
vb <0
320
va < 0
329
vb <0
378
va < 0
1015
vb <0
1133
va <

va < 0
433
vb <0
446
va < 0
520
va < 0
536
vc < 0
729
vc < 0
879
vb <0
978
vb <0
989
vc < 0
1037
vc < 0
1039
vd < 0
1160
vc < 0
1162
1314
va < 0
-1
1384
vb <0
-1
21
va < 0
133
va < 0
139
va < 0
142
va < 0
294
va < 0
331
va < 0
375
vb <0
395
vb <0
471
vb <0
549
va < 0
583
vb <0
715
vb <0
903
vb <0
953
va < 0
995
vb <0
1108
va < 0
1206
vb <0
-1
19
va < 0
255
vb <0
281
va < 0
464
va < 0
841
vc < 0
1048
vb <0
1114
vb <0
1163
va < 0
1163
va < 0
1298
vb <0
1302
vb <0
1315
vb <0
1331
va < 0
1333
va < 0
1448
va < 0
-1
85
vb <0
253
va < 0
330
vb <0
369
va < 0
489
va < 0
545
va < 0
730
va < 0
874
vb <0
945
va < 0
963
vb <0
1031
vc < 0
1052
vc < 0
1052
vc < 0
1123
va < 0
1191
va < 0
1309
va < 0
1329
vb <0
1334
va < 0
-1
129
vb <0
721
va < 0
876
va < 0
885
va < 0
968
va < 0
1014
va < 0
1245
vb <0
1247
vb <0
1312
va < 0
-1
62
va < 0
175
vd < 0
1085
va < 0
1245
va < 0
1341
va < 0
1406
vc < 0
-1
16
vb <0
354
va < 0
487
va < 0
516
va < 0
577
va < 0
684
vc < 0
783
va < 0
1064
va < 0
-1
198
va < 0
405
v

va < 0
1107
va < 0
1263
va < 0
1269
va < 0
1272
vb <0
1466
va < 0
1468
va < 0
-1
138
vb <0
160
va < 0
211
vb <0
254
va < 0
280
va < 0
282
va < 0
300
vc < 0
309
va < 0
357
va < 0
357
va < 0
367
vb <0
373
va < 0
389
vd < 0
390
vb <0
407
vb <0
457
vb <0
622
va < 0
674
vb <0
698
va < 0
717
va < 0
723
va < 0
723
va < 0
736
vc < 0
784
va < 0
841
vb <0
844
va < 0
945
va < 0
977
va < 0
985
va < 0
1005
va < 0
1013
va < 0
1053
va < 0
1113
vc < 0
1120
vb <0
1181
va < 0
1183
va < 0
1235
vc < 0
1322
vc < 0
1322
vc < 0
1388
vc < 0
1436
va < 0
1440
va < 0
1441
vb <0
1454
vb <0
-1
516
va < 0
638
vb <0
651
va < 0
652
vb <0
799
vb <0
800
vb <0
-1
330
vb <0
854
va < 0
1119
vb <0
-1
171
va < 0
284
vb <0
288
va < 0
307
va < 0
399
va < 0
519
va < 0
562
va < 0
807
vb <0
810
vb <0
965
va < 0
1165
va < 0
1255
vb <0
1413
va < 0
1414
vb <0
1415
va < 0
-1
113
vb <0
532
vd < 0
629
va < 0
781
vb <0
1031
vc < 0
1231
vb <0
1262
va < 0
1313
va < 0
1362
vc < 0
1363
vb <0
1369
va < 0
-1
9
vb <0
272
vb <0
956
vb <0
1136


292
vb <0
-1
111
vb <0
374
va < 0
685
va < 0
1150
va < 0
1482
va < 0
-1
99
va < 0
123
vb <0
758
vb <0
1077
va < 0
1464
va < 0
-1
1108
vd < 0
-1
61
va < 0
86
va < 0
90
vc < 0
238
va < 0
472
va < 0
604
va < 0
997
va < 0
1105
va < 0
1261
vd < 0
1270
va < 0
1271
vb <0
1344
vb <0
1347
va < 0
-1
44
vd < 0
290
vc < 0
332
vb <0
416
va < 0
552
vb <0
565
va < 0
566
vc < 0
745
vd < 0
932
vb <0
969
vb <0
970
vd < 0
1032
va < 0
1084
vb <0
1170
vc < 0
1203
vc < 0
1216
vb <0
1296
vd < 0
1301
vd < 0
1303
va < 0
1316
va < 0
1439
vc < 0
-1
320
va < 0
378
va < 0
801
va < 0
-1
172
va < 0
471
vb <0
476
vd < 0
484
va < 0
491
vd < 0
522
vb <0
670
vb <0
772
vb <0
802
vb <0
814
vd < 0
897
va < 0
903
vb <0
949
vd < 0
994
vc < 0
1046
vc < 0
1202
vd < 0
1320
vb <0
1321
vb <0
1337
vc < 0
-1
58
va < 0
415
va < 0
447
va < 0
629
va < 0
771
vc < 0
781
va < 0
913
va < 0
922
va < 0
936
va < 0
1158
va < 0
1351
va < 0
1359
vb <0
1366
vc < 0
1370
va < 0
-1
607
va < 0
899
vc < 0
1324
va < 0
1409
va < 0
-1
175
vb <0
477
vb <

...

va < 0
1247
va < 0
1261
va < 0
1271
vc < 0
1344
vc < 0
1347
vb <0
-1
277
va < 0
319
va < 0
342
vb <0
512
va < 0
688
vc < 0
1300
vb <0
-1
516
va < 0
599
va < 0
638
vb <0
673
va < 0
-1
46
vc < 0
277
va < 0
762
va < 0
1278
vb <0
-1
314
va < 0
630
vb <0
678
va < 0
-1
17
va < 0
391
vb <0
396
vb <0
602
va < 0
1025
va < 0
1371
va < 0
-1
20
va < 0
73
vb <0
74
va < 0
108
va < 0
505
va < 0
651
va < 0
759
va < 0
798
va < 0
799
vb <0
800
vb <0
801
va < 0
803
vc < 0
-1
81
va < 0
218
vc < 0
232
vb <0
263
vd < 0
283
vb <0
287
vb <0
587
vb <0
771
va < 0
780
va < 0
868
886
vc < 0
951
va < 0
1010
va < 0
1065
vb <0
1173
va < 0
1366
va < 0
-1
42
vb <0
119
va < 0
192
vb <0
241
va < 0
359
vb <0
430
va < 0
458
va < 0
710
va < 0
777
vc < 0
1009
vb <0
1021
vb <0
1160
vc < 0
1193
va < 0
1264
vd < 0
1450
va < 0
-1
48
va < 0
112
va < 0
118
va < 0
174
vb <0
227
va < 0
258
va < 0
455
vc < 0
470
va < 0
632
vc < 0
677
vc < 0
712
va < 0
793
va < 0
845
va < 0
-1
156
vb <0
404
va < 0
995
vb <0
1082
va < 0
1108
vb <0
-1


vb <0
290
vc < 0
347
vb <0
435
va < 0
454
vb <0
556
vc < 0
603
vb <0
615
vb <0
1032
vc < 0
1216
vb <0
-1
180
va < 0
444
va < 0
538
va < 0
668
vc < 0
836
vb <0
931
va < 0
1041
va < 0
1281
va < 0
1292
vb <0
-1
503
-1
59
va < 0
116
vb <0
127
va < 0
153
va < 0
183
va < 0
297
va < 0
309
va < 0
389
vd < 0
495
vb <0
498
vb <0
612
vb <0
1047
va < 0
1120
vb <0
1152
va < 0
1319
va < 0
1434
va < 0
1435
vb <0
1436
va < 0
1438
va < 0
1440
va < 0
1441
vb <0
1451
va < 0
1454
vb <0
-1
278
va < 0
674
vb <0
802
va < 0
898
vb <0
1145
va < 0
1328
va < 0
1400
va < 0
1403
va < 0
-1
20
vb <0
185
va < 0
324
va < 0
344
va < 0
709
vb <0
759
va < 0
799
va < 0
1139
va < 0
-1
25
vb <0
69
vb <0
121
vb <0
190
va < 0
234
vb <0
435
va < 0
537
vb <0
556
vc < 0
634
vb <0
777
vc < 0
819
vd < 0
932
vc < 0
1089
va < 0
1102
va < 0
1255
va < 0
1264
vd < 0
1281
va < 0
-1
264
va < 0
459
va < 0
892
va < 0
-1
110
va < 0
134
va < 0
429
vb <0
446
va < 0
448
vb <0
666
vc < 0
862
va < 0
978
vb <0
1037
vc < 0
1039
vd < 0
1112
vb <0
1

va < 0
1452
va < 0
1482
vb <0
-1
152
va < 0
229
vb <0
232
vb <0
267
va < 0
318
vb <0
403
va < 0
481
va < 0
544
va < 0
667
va < 0
747
vc < 0
797
va < 0
959
va < 0
984
vb <0
1075
va < 0
1085
va < 0
1100
va < 0
1107
va < 0
1217
va < 0
1466
va < 0
-1
125
vc < 0
193
vb <0
281
va < 0
469
vb <0
494
vb <0
629
va < 0
656
va < 0
1055
vb <0
1163
va < 0
1163
va < 0
1242
va < 0
1294
va < 0
-1
1187
va < 0
1318
va < 0
1331
va < 0
1334
va < 0
1383
va < 0
-1
52
va < 0
128
vc < 0
198
va < 0
555
va < 0
560
vb <0
964
va < 0
1197
vb <0
1343
vc < 0
-1
51
va < 0
-1
237
vb <0
1488
vb <0
-1
13
va < 0
130
vb <0
177
vc < 0
240
va < 0
347
va < 0
746
vc < 0
877
vc < 0
881
vb <0
967
vb <0
1055
vb <0
1191
vb <0
-1
342
vb <0
355
vc < 0
371
vb <0
388
vb <0
509
va < 0
1056
vc < 0
1299
va < 0
1408
vc < 0
1415
va < 0
1489
vb <0
-1
50
vb <0
197
vc < 0
305
va < 0
-1
237
va < 0
695
va < 0
1490
va < 0
-1
8
va < 0
14
vb <0
90
vc < 0
103
va < 0
129
vc < 0
182
va < 0
376
vc < 0
383
va < 0
461
va < 0
472
va < 0
485
va < 0
501
va

vb <0
1231
vb <0
1262
va < 0
1313
va < 0
1362
vc < 0
1363
vb <0
1369
va < 0
-1
81
va < 0
287
vb <0
447
vc < 0
460
va < 0
532
vd < 0
780
va < 0
1030
vb <0
1065
vc < 0
1158
va < 0
1307
va < 0
1351
vb <0
1354
va < 0
1357
va < 0
1359
vb <0
1361
vc < 0
1364
vb <0
1374
vb <0
-1
187
va < 0
225
va < 0
372
vb <0
692
vb <0
695
va < 0
696
va < 0
1373
va < 0
1413
va < 0
1414
va < 0
1489
vb <0
1491
va < 0
-1
42
va < 0
241
va < 0
430
va < 0
450
va < 0
710
vb <0
777
vc < 0
1009
vb <0
1021
va < 0
1038
va < 0
1193
va < 0
1264
vb <0
1277
va < 0
1450
va < 0
1488
vb <0
-1
102
va < 0
202
va < 0
340
vc < 0
480
va < 0
575
vb <0
608
vc < 0
788
vb <0
831
va < 0
924
vb <0
1043
va < 0
1072
va < 0
1166
vb <0
1447
va < 0
-1
379
vb <0
398
va < 0
-1
79
va < 0
136
va < 0
295
va < 0
397
va < 0
412
vb <0
568
va < 0
568
va < 0
581
vb <0
582
vb <0
605
va < 0
606
va < 0
669
vb <0
669
vb <0
748
vb <0
909
vb <0
910
vb <0
910
vb <0
911
va < 0
912
va < 0
-1
54
va < 0
149
va < 0
194
va < 0
351
vb <0
414
va < 0
651
va < 0
672
v

577
va < 0
639
vb <0
645
va < 0
684
va < 0
773
va < 0
783
vb <0
934
va < 0
1064
vb <0
-1
292
vb <0
-1
455
vb <0
1196
vc < 0
1241
vb <0
1342
vb <0
1430
vb <0
1431
va < 0
-1
148
va < 0
279
vb <0
604
va < 0
662
vb <0
721
vc < 0
865
vc < 0
935
vd < 0
997
vb <0
1044
va < 0
1060
vc < 0
1061
va < 0
1080
va < 0
1247
va < 0
1261
va < 0
1271
vc < 0
1306
vb <0
-1
190
va < 0
417
va < 0
923
vc < 0
930
va < 0
1051
vd < 0
1057
va < 0
1084
vb <0
1143
vb <0
1453
va < 0
-1
61
va < 0
86
va < 0
90
vc < 0
238
va < 0
472
va < 0
604
va < 0
997
va < 0
1105
va < 0
1261
vd < 0
1270
va < 0
1271
vb <0
1344
vb <0
1347
va < 0
-1
10
vb <0
72
vc < 0
204
vc < 0
234
va < 0
450
va < 0
634
vb <0
714
vb <0
839
va < 0
894
vc < 0
982
vc < 0
1063
va < 0
1102
vc < 0
1105
vd < 0
1264
vc < 0
1280
va < 0
1405
va < 0
1417
vb <0
-1
263
va < 0
423
vc < 0
559
va < 0
951
vb <0
1002
vb <0
1153
va < 0
1167
vd < 0
1180
vb <0
1195
vb <0
1470
vb <0
-1
135
va < 0
250
va < 0
499
va < 0
576
vb <0
-1
1407
-1
15
va < 0
106
vb <0
147
vb <0
167


In [161]:
print(mask2)

(array([  -1,   -1,   -1, ...,  134, 1291,   -1]), array([[0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       ...,
       [0.05885953, 0.29844447, 0.18160339, 0.46109261],
       [0.11787258, 0.20210807, 0.67307694, 0.0069424 ],
       [0.        , 0.        , 0.        , 0.        ]]))


In [15]:
aabbindx = mesh2.pos_to_aabb_index(points)
mesh2.aabb[314,:]

array([ 747, 1100,   -1, ...,   -1,   -1,   -1])

In [19]:
mesh2.nodes[mesh2.tetra[1100]]

array([[0.40625   , 0.        , 0.2578125 ],
       [0.36230469, 0.        , 0.4140625 ],
       [0.50612419, 0.        , 0.35839043],
       [0.48026525, 0.21744951, 0.35090743]])

In [18]:
points[0,:]

array([0.44240712, 0.17424164, 0.31584794])

In [ ]:
points = mesh2.nodes[mesh2.tetra]
print(points.shape)

vab = points[ :, 1, :] - points[ :, 0, :]
vac = points[ :, 2, :] - points[ :, 0, :]
vad = points[ :, 3, :] - points[ :, 0, :]
v = np.einsum('ij, ij->i', vab, np.cross(vac, vad, axisa=1, axisb=1)) / 6.
plt.hist(v)

In [ ]:
mesh2.step[0]*mesh2.step[1]*mesh2.step[2]

In [ ]:
#%%snakeviz
mask = mesh2.get_tetra2(points)#np.random.random((100,3)))


In [ ]:
print(np.sum(mask2[0] < 0))

In [ ]:
points[mask2 == -1]

In [ ]:
np.sum(mask[0] == -1)

In [ ]:
from LoopStructural.supports.tet_mesh import TetMesh

In [ ]:
mesh = TetMesh()
mesh.setup_mesh(np.array([[0,0,0],[1,1,1]]),n_tetra=3000)

In [ ]:
mesh.elements_for_array(points)

In [8]:
xi = 8
yi = 8
zi = 8
nx=9
ny=9
nz=9

In [9]:
global_index = xi+yi*nx+zi*nx*ny

In [10]:
print(global_index)

728


In [11]:
9*9*9

729

In [ ]:
9*9*9